# Protected Area Boundary Change

This notebook outlines the methodology used to measure at protected area boundaries via GEE. 

The notebook queries MODIS imagery and returns the gradient values of relevant bands as well as the vegetation indices NDVI and EVI. This code expects an annual time span and returns a geoTIFF for each band in each park for that year.

In [ ]:
from utils import *
from config import *
import pandas as pd
from datetime import datetime

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='dse-staff')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

## Class Definitions

In [ ]:
def main(wdpaid, year, show_map=False, band_name=None):
    """Main function to process protected area boundary analysis"""
    # Initialize classes
    geo_ops = GeometryOperations()
    img_ops = ImageOperations()
    stats_ops = StatsOperations()
    viz = Visualization()
    feature_processor = FeatureProcessor(geo_ops, img_ops, stats_ops)
    exporter = ExportResults()

    # Load and process protected area geometry
    pa = load_local_data(wdpaid)
    pa_geometry = pa.geometry()
    aoi = geo_ops.buffer_polygon(pa_geometry)
    aoi = geo_ops.mask_water(aoi)
    aoi_with_biome = geo_ops.get_biome(aoi)

    # Process imagery and add indices
    modis_ic = img_ops.modis.filter(img_ops.filter_for_year(aoi, year))
    band_names = modis_ic.first().bandNames()
    composite = modis_ic.reduce(ee.Reducer.median()).rename(band_names).clip(aoi)
    image = img_ops.add_indices_to_image(composite)

    # Process features and collect statistics
    feature_info = feature_processor.collect_feature_info(pa, aoi_with_biome)
    computed_stats = feature_processor.process_all_bands(image, pa_geometry, aoi)
    all_stats = feature_processor.compile_statistics(feature_info, computed_stats, year)
    
    # Save results
    df = pd.DataFrame(all_stats)
    exporter.save_df_to_gcs(df, 'dse-staff', wdpaid, year)

    # Visualization
    if show_map:
        band_stats = next(cs for cs in computed_stats if cs["band_name"] == band_name)
        Map = viz.create_map(pa_geometry, band_stats['buffer_pixels'], band_stats['boundary_pixels'])

    return print("Analysis ")

In [ ]:
#Map = main("916", 2010) #show_map=True, band_name="EVI")
#Map

In [ ]:
import concurrent.futures

def run_main_for_years(wdpaid, start_year, n_years, show_map=False, band_name=None, max_workers=4):
    """
    Runs main() in parallel for a sequence of years.
    Args:
        wdpaid: Protected area ID
        start_year: First year (int)
        n_years: Number of years to run (int)
        show_map: Whether to show map (default False)
        band_name: Band name for visualization (default None)
        max_workers: Number of parallel workers
    Returns:
        List of results from main()
    """
    years = [start_year + i for i in range(n_years)]
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_year = {
            executor.submit(main, wdpaid, year, show_map, band_name): year for year in years
        }
        for future in concurrent.futures.as_completed(future_to_year):
            year = future_to_year[future]
            try:
                result = future.result()
                results.append((year, result))
            except Exception as exc:
                print(f"Year {year} generated an exception: {exc}")
    # Sort results by year
    results.sort(key=lambda x: x[0])
    return results

In [ ]:
run_main_for_years("916", 2001, 10), max_workers=10)

-add write out for each step, with identifier for each park
-ray to run in parallel in python
-use glance to check usage